# Fine-tuning BERT (base or large) on a Token-Classification (NER) task by using the library adapter-transformers (script version)

- **Credit**: [Hugging Face](https://huggingface.co/) and [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers)
- **Author**: [Pierre GUILLOU](https://www.linkedin.com/in/pierreguillou/)
- **Date**: 02/07/2021
- **Blog post**: []()
- **Link to the folder in github with this notebook and all necessary scripts**: [token-classification with adapters](https://github.com/piegu/language-models/tree/master/adapters/token-classification/)
- **Link to the adapters in the AdapterHub**: 

## 1. Context

### Objective

The objective here is to **fine-tune a Masked Language Model (MLM) like BERT (base or large) for a Token-Classification (NER) task by training adapters (library [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers)), not the embeddings and transformers layers of the MLM model**, and to compare results with BERT model fully fine-tune for the same task.

The interest is obvious: if you need models for different NLP tasks, instead of fine-tuning and storing one model by NLP task, **you store only one MLM model and the trained tasks adapters which sizes are about 3% of the MLM model one**. More, the loading of these adapters in production is very easy.

### Content

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a Token-Classification (NER) task, which is the task of predicting a label for each token. We will use the library [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers) and see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

![Widget inference representing the QA task](images/token_classification_adapter.png)

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

### History and Credit

This notebook is an adaptation of the following notebooks and scripts for **fine-tuning a (transformer) Masked Language Model (MLM) like BERT (base or large) on the Token-Classification (NER) task with any NER dataset** (we use here the [LeNER-br dataset](https://huggingface.co/datasets/lener_br):
- **from [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers)** | notebook [07a_Adapter_train_NER_with_id2label.ipynb](https://github.com/Adapter-Hub/adapter-transformers/blob/master/notebooks/07a_Adapter_train_NER_with_id2label.ipynb) and script [run_ner.py](https://github.com/Adapter-Hub/adapter-transformers/blob/master/examples/token-classification/run_ner.py) (this script was adapted from the script [run_qa.py](https://github.com/huggingface/transformers/blob/master/examples/pytorch/token-classification/run_ner.py) of HF)
- **from [transformers](https://github.com/huggingface/transformers) of Hugging Face** | notebook [token_classification.ipynb](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb) and script [run_ner.py](https://github.com/huggingface/transformers/blob/master/examples/pytorch/token-classification/run_ner.py) 

In order to speed up the fine-tuning of the model on only one GPU, the library [DeepSpeed](https://www.deepspeed.ai/) could be used by applying the configuration provided by HF in the notebook [transformers + deepspeed CLI](https://github.com/stas00/porting/blob/master/transformers/deepspeed/DeepSpeed_on_colab_CLI.ipynb) but as the library adapter-transformers is not synchronized with the last version of the library transformers of HF, we keep that option for the future.

### Major changes from original notebooks and scripts

The notebook [token_classification.ipynb](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb) and the script [run_ner.py](https://github.com/Adapter-Hub/adapter-transformers/blob/master/examples/token-classification/run_ner.py) allow to evaluate the model performance against f1 metric at the end of each epoch, and not against validation loss.

This is very important as we consider the metric when selecting a model, not the loss. Therefore, we decided to update this script inside this notebook (by simulating terminal command line) instead of running code in cells as done in the notebook of HF.

Therefore, we updated the notebook [token_classification.ipynb](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb) to [token_classification_adapter.ipynb](https://github.com/piegu/language-models/blob/master/adapters/token_classification/token_classification_adapter.ipynb) with the following changes:
- **EarlyStopping** by selecting the model with the highest eval f1 (patience of 3 before ending the training)
- **MAD-X 2.0** that allows not to train adapters in the last transformer layer (read page 6 of [UNKs Everywhere: Adapting Multilingual Language Models to New Scripts](https://arxiv.org/pdf/2012.15562.pdf))
- **Stack method** for the lang and task adapters when a lang adapter is loaded ([doc](https://docs.adapterhub.ml/adapter_composition.html?highlight=stack#stack))

## 2. Installation

In [1]:
import pathlib
from pathlib import Path

#root path
root = Path.cwd()

In [2]:
import pickle
import pandas as pd

In [3]:
import sys; print('python:',sys.version)

import torch; print('Pytorch:',torch.__version__)

import transformers; print('adapter-transformers:',transformers.__version__)
import transformers; print('HF transformers:',transformers.__hf_version__)
import tokenizers; print('tokenizers:',tokenizers.__version__)
import datasets; print('datasets:',datasets.__version__)

# import deepspeed; print('deepspeed:',deepspeed.__version__)

# Versions used in the virtuel environment of this notebook:

# python: 3.8.10 (default, Jun  4 2021, 15:09:15) 
# [GCC 7.5.0]
# Pytorch: 1.9.0
# adapter-transformers: 2.0.1
# transformers: 4.5.1
# tokenizers: 0.10.3
# datasets: 1.8.0

python: 3.8.10 (default, Jun  4 2021, 15:09:15) 
[GCC 7.5.0]
Pytorch: 1.9.0
adapter-transformers: 2.0.1
HF transformers: 4.5.1
tokenizers: 0.10.3
datasets: 1.8.0


## 3. Model & dataset

In [4]:
# Select a MLM BERT base or large in the dataset language
model_checkpoint = "neuralmind/bert-base-portuguese-cased"
# model_checkpoint = "neuralmind/bert-large-portuguese-cased"

# LeNER-br in Portuguese
dataset_name = "lener_br"

## 4. Main hyperparameters

In [5]:
task = "ner"

In [74]:
# training arguments
batch_size = 16
gradient_accumulation_steps = 1

learning_rate = 1e-4
num_train_epochs = 10.
early_stopping_patience = 3

adam_epsilon = 1e-6

fp16 = True
ds = False # If True, we use DeepSpeed

# best model
load_best_model_at_end = True 
metric_for_best_model = "f1"
greater_is_better = True

In [7]:
# train adapter
train_adapter = True # we want to train an adapter
load_adapter = None # we do not upload an existing adapter 

# lang adapter
with_adapters_mlm = False # if False, we do not upload an existing lang adapter

if with_adapters_mlm:
    # define values to upload a specific lang adapter
    num_train_epochs_mlm = 100.0
    adam_epsilon_mlm = adam_epsilon
    adapter_composition = "stack" # we will stack the lang and task adapters
else:
    adapter_composition = None

# if True, do not put adapter in the last transformer layer
madx2 = True

## 5. Configuration

### GPU

In [8]:
# gpu
n_gpu = 1 # train on just one GPU
gpu = 0 # select the GPU

In [9]:
# Select GPU 0
import os
os.environ['MASTER_ADDR'] = 'localhost'
if gpu == 0:
    os.environ['MASTER_PORT'] = '9996' # modify if RuntimeError: Address already in use # GPU 0
elif gpu == 1:
    os.environ['MASTER_PORT'] = '9995'
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = str(gpu)
os.environ['WORLD_SIZE'] = "1"

### Training arguments of the HF trainer

In [10]:
# setup the training argument
do_train = True 
do_eval = True 

# if you want to test the trainer, set up the following variables
max_train_samples = 200 # None
max_val_samples = 50 # None

# epochs, bs, GA
evaluation_strategy = "epoch" 

# fp16
fp16_opt_level = 'O1'
fp16_backend = "auto"
fp16_full_eval = False

# optimizer (AdamW)
weight_decay = 0.01 # 0.0
adam_beta1 = 0.9
adam_beta2 = 0.999

# scheduler
lr_scheduler_type = 'linear'
warmup_ratio = 0.0
warmup_steps = 0

# logs
logging_strategy = "steps"
logging_first_step = True # False
logging_steps = 500     # if strategy = "steps"
eval_steps = logging_steps # logging_steps

# checkpoints
save_strategy = "epoch" # steps
save_steps = 500 # if save_strategy = "steps"
save_total_limit = 1 # None

# no cuda, seed
no_cuda = False
seed = 42

# bar
disable_tqdm = False # True
remove_unused_columns = True

In [11]:
# folder for training outputs

outputs = model_checkpoint.replace('/','-') + '_' + dataset_name + '/'
if with_adapters_mlm:
    outputs = outputs + 'mlm_' + str(task) + '_AdCompo' + str(adapter_composition)
else:
    outputs = outputs + str(task)
outputs = outputs \
+ '_lr' + str(learning_rate) \
+ '_bs' + str(batch_size) \
+ '_eps' + str(adam_epsilon) \
+ '_epochs' + str(num_train_epochs) \
+ '_wamlm' + str(with_adapters_mlm) \
+ '_madx2' + str(madx2) \
+ '_ds' + str(ds) \
+ '_fp16' + str(fp16) \
+ '_best' + str(load_best_model_at_end) \
+ '_metric' + str(metric_for_best_model)

# path to outputs
path_to_outputs = root/'models_outputs'/outputs

# subfolder for model outputs
output_dir = path_to_outputs/'output_dir' 
overwrite_output_dir = True # False

# logs
logging_dir = path_to_outputs/'logging_dir'

### Adapters config

#### Task adapter

In [12]:
# task adapter config
adapter_config = "pfeiffer" # houlsby is possible, too
adapter_non_linearity = 'gelu' # relu is possible, too
adapter_reduction_factor = 16
language = 'pt' # pt = Portuguese

#### Lang adapter

In [38]:
if with_adapters_mlm:
    
    # hyperparameters used for fine-tuning the MLM with lang adapter
    learning_rate_mlm = 1e-4
    batch_size_mlm = 32
    gradient_accumulation_steps_mlm = 1
    adam_epsilon_mlm = 1e-6
    num_train_epoch_mlms = 100.
    madx2_mlm = madx2
    ds_mlm = False
    fp16_mlm = True
    load_best_model_at_end_mlm = True
    metric_for_best_model_mlm = "loss"
    
    # path to lang adapter
    outputs_mlm = model_checkpoint.replace('/','-') + '_' + dataset_name + '/mlm' \
    + '_lr' + str(learning_rate_mlm) \
    + '_bs' + str(batch_size_mlm) \
    + '_GAS' + str(gradient_accumulation_steps_mlm) \
    + '_eps' + str(adam_epsilon_mlm) \
    + '_epochs' + str(num_train_epoch_mlms) \
    + '_madx2' + str(madx2_mlm) \
    + '_ds' + str(ds_mlm) \
    + '_fp16' + str(fp16_mlm) \
    + '_best' + str(load_best_model_at_end_mlm) \
    + '_metric' + str(metric_for_best_model_mlm)

    path_to_outputs = root/'models_outputs'/outputs_mlm
    
    # Config of the lang adapter
    lang_adapter_path = path_to_outputs/'adapters-mlm/'
    
    load_lang_adapter = lang_adapter_path
    lang_adapter_config = str(lang_adapter_path + "/adapter_config.json")
    lang_adapter_non_linearity = 'gelu'
    lang_adapter_reduction_factor = 2
    language_mlm = language
else:
    load_lang_adapter = None

## 6. Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [14]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [lener_br dataset](https://huggingface.co/datasets/lener_br). The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [ ]:
datasets = load_dataset(dataset_name)

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'ner_tags', 'tokens'],
        num_rows: 7828
    })
    validation: Dataset({
        features: ['id', 'ner_tags', 'tokens'],
        num_rows: 1177
    })
    test: Dataset({
        features: ['id', 'ner_tags', 'tokens'],
        num_rows: 1390
    })
})

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

In [17]:
datasets["train"][0]

{'id': '0',
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['EMENTA',
  ':',
  'APELAÇÃO',
  'CÍVEL',
  '-',
  'AÇÃO',
  'DE',
  'INDENIZAÇÃO',
  'POR',
  'DANOS',
  'MORAIS',
  '-',
  'PRELIMINAR',
  '-',
  'ARGUIDA',
  'PELO',
  'MINISTÉRIO',
  'PÚBLICO',
  'EM',
  'GRAU',
  'RECURSAL',
  '-',
  'NULIDADE',
  '-',
  'AUSÊNCIA',
  'DE',
  'INTERVENÇÃO',
  'DO',
  'PARQUET',
  'NA',
  'INSTÂNCIA',
  'A',
  'QUO',
  '-',
  'PRESENÇA',
  'DE',
  'INCAPAZ',
  '-',
  'PREJUÍZO',
  'EXISTENTE',
  '-',
  'PRELIMINAR',
  'ACOLHIDA',
  '-',
  'NULIDADE',
  'RECONHECIDA',
  '.']}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [18]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=13, names=['O', 'B-ORGANIZACAO', 'I-ORGANIZACAO', 'B-PESSOA', 'I-PESSOA', 'B-TEMPO', 'I-TEMPO', 'B-LOCAL', 'I-LOCAL', 'B-LEGISLACAO', 'I-LEGISLACAO', 'B-JURISPRUDENCIA', 'I-JURISPRUDENCIA'], names_file=None, id=None), length=-1, id=None)

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:
- 'PER' for person
- 'ORG' for organization
- 'LOC' for location
- 'MISC' for miscellaneous

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [19]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O',
 'B-ORGANIZACAO',
 'I-ORGANIZACAO',
 'B-PESSOA',
 'I-PESSOA',
 'B-TEMPO',
 'I-TEMPO',
 'B-LOCAL',
 'I-LOCAL',
 'B-LEGISLACAO',
 'I-LEGISLACAO',
 'B-JURISPRUDENCIA',
 'I-JURISPRUDENCIA']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [20]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [21]:
show_random_elements(datasets["train"])

,id,ner_tags,tokens
0,6042,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[§, 2o, O, descumprimento, de, qualquer, dos, incisos, deste, artigo, ou, de, qualquer, obrigação, contida, no, termo, de, compromisso, caracteriza, vínculo, de, emprego, do, educando, com, a, parte, concedente, do, estágio, para, todos, os, fins, da, legislação, trabalhista, e, previdenciária, .]"
1,292,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[Caberia, ao, pregoeiro, convocar, os, licitantes, ,, sucessivamente, ,, conforme, as, respectivas, classificações, ,, e, conceder-lhes, a, oportunidade, de, comprovar, a, viabilidade, financeira, de, suas, propostas, .]"
2,3865,"[O, O, O, O, O, O, O, O, O, O, O, O, B-TEMPO, O, B-TEMPO, O, O]","[No, entanto, ,, a, Apelação, foi, protocolizada, em, Juízo, apenas, no, dia, 29/6/2017, (, quinta-feira, ), .]"
3,6909,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[Por, conseguinte, ,, nada, há, a, ser, reformado, no, julgado, recorrido, ,, ficando, mantida, a, multa, aplicada, na, oportunidade, em, que, foram, apreciados, os, embargos, declaratórios, ., '']"
4,4456,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, teor, do, acórdão, embargado, evidencia, a, desnecessidade, de, integração, ,, mostrando-se, o, aresto, claro, ,, coerente, e, livre, de, omissão, ,, pois, examinou, as, questões, propostas, de, acordo, com, o, entendimento, deste, Tribunal, .]"
5,4714,"[O, O, O, O, O, O, O, O, O, O, B-JURISPRUDENCIA, I-JURISPRUDENCIA, I-JURISPRUDENCIA, O, O, O, O, B-PESSOA, I-PESSOA, O, B-ORGANIZACAO, I-ORGANIZACAO, O, O, O, B-TEMPO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORGANIZACAO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORGANIZACAO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORGANIZACAO, I-ORGANIZACAO, I-ORGANIZACAO, I-ORGANIZACAO, O, ...]","[2, ..., Agravo, regimental, a, que, se, nega, provimento, (, AI, nº, 548.235/SP-AgR, ,, Relator, o, Ministro, Eros, Grau, ,, Segunda, Turma, ,, DJ, de, 9/6/06, -, grifamos, e, destacamos, ), Nesse, exato, sentido, ,, julgando, pedido, relativo, à, complementação, de, aposentadoria, em, que, ,, na, ocasião, ,, o, TST, afirmou, a, competência, da, Justiça, do, Trabalho, em, razão, da, natureza, celetista, da, relação, de, emprego, ,, o, e., STF, reformou, o, julgado, para, afirmar, sua, jurisprudência, :, Ao, indeferir, o, processamento, do, presente, Recurso, Extraordinário, ,, assim, fundamentou, sua, decisão, o, ilustre, Presidente, do, Tribunal, Superior, do, Trabalho, ..., ...]"
6,5074,"[O, O, O, O]","[HOMICÍDIO, QUALIFICADO, TENTADO, .]"
7,3164,"[O, O, O, O, O, B-JURISPRUDENCIA, I-JURISPRUDENCIA, I-JURISPRUDENCIA, I-JURISPRUDENCIA, I-JURISPRUDENCIA, I-JURISPRUDENCIA, O, O, O, B-PESSOA, I-PESSOA, O, O, O, B-TEMPO, O, O, O, O, O, O, O]","[(, ..., ), '', (, Recurso, Extraordinário, com, Agravo, n°, 674.299/PR, ,, Relator, Ministro, Dias, Toffoli, ,, julgado, em, 27/2/2013, ), '', EMENTA, :, HABEAS, CORPUS, .]"
8,3008,"[O, O, O, O, O, O]","[A, preliminar, merece, ser, acolhida, .]"
9,3487,"[O, O, O, O, O, O, O, O, O]","[Réu, menor, ,, primário, ,, de, bons, antecedentes, .]"


## 7. Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [22]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [23]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [24]:
tokenizer("O termo de compromisso deverá ser firmado pelo estagiário.")

{'input_ids': [101, 231, 2476, 125, 11798, 11588, 333, 16315, 243, 423, 418, 2245, 488, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [25]:
tokenizer(["O", "termo", "de", "compromisso", "deverá", "ser", "firmado", "pelo", "estagiário", "."], is_split_into_words=True)

{'input_ids': [101, 231, 2476, 125, 11798, 11588, 333, 16315, 243, 423, 418, 2245, 488, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [26]:
example = datasets["train"][4]
print(example["tokens"])

['-', 'Preliminar', 'acolhida', 'para', 'reconhecer', 'a', 'nulidade', 'da', 'sentença', '.']


In [27]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '-', 'Pre', '##lim', '##inar', 'acolh', '##ida', 'para', 'reconhecer', 'a', 'nu', '##li', '##dade', 'da', 'sentença', '.', '[SEP]']


Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [28]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(10, 17)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [29]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 1, 2, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [30]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

17 17


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [31]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [32]:
def tokenize_and_align_labels(examples):
#     tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [33]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 192, 7463, 8427, 22301, 131, 12127, 9008, 22301, 22402, 16484, 187, 22360, 22339, 9008, 118, 177, 22402, 16484, 10836, 13760, 7545, 22320, 22323, 22351, 22301, 22402, 16484, 212, 8718, 250, 7665, 6072, 213, 8718, 22301, 6538, 118, 11635, 9008, 13270, 7073, 6765, 118, 11741, 22328, 22341, 6392, 22301, 212, 9008, 22317, 213, 7073, 6538, 22321, 22352, 21748, 22317, 212, 22371, 22318, 22327, 6162, 22317, 192, 22311, 278, 5650, 22341, 257, 5476, 15289, 5903, 22327, 118, 248, 18199, 6392, 11836, 22309, 118, 177, 10409, 22420, 22320, 14298, 22301, 10836, 13760, 16017, 22322, 22339, 12547, 22402, 16484, 15040, 18868, 22322, 22349, 22341, 9208, 248, 22301, 13760, 11846, 22379, 22320, 14298, 22301, 177, 5226, 22341, 22317, 118, 11635, 3341, 12547, 22402, 22301, 10836, 13760, 4529, 5869, 22351, 118, 11635, 22309, 22333, 22341, 22360, 22351, 22317, 192, 22348, 6538, 16017, 8427, 22309, 118, 11635, 9008, 13270, 7073, 6765, 11247, 7918, 22340, 6392, 22301, 118, 248, 18199, 6392,

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [34]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## 8. Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [89]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model check

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

In [90]:
# number of model parameters
model_num_param=0
for p in model.parameters():
    model_num_param+=p.numel()
model_num_param

108342541

## 9. Adapters

In [76]:
# Setup adapters
from transformers.adapters.composition import Stack

if train_adapter:
        
    # new
    if madx2:
        # do not add adapter in the last transformer layers 
        leave_out = [len(model.bert.encoder.layer)-1]
    else:
        leave_out = []
        
    # new
    # task_name = data_args.dataset_name or "mlm"
    task_name = "ner"
    
    # optionally load another pre-trained language adapter
    if load_lang_adapter:
        # resolve the language adapter config
        lang_adapter_config = AdapterConfig.load(
                lang_adapter_config,
                non_linearity=lang_adapter_non_linearity,
                reduction_factor=lang_adapter_reduction_factor,
                leave_out=leave_out,
            )
        # load the language adapter from Hub
        task_mlm_load_as = 'mlm'
        lang_adapter_name = model.load_adapter(
                load_lang_adapter,
                config=lang_adapter_config,
                load_as=task_mlm_load_as,
                with_head = False
            )
    else:
        lang_adapter_name = None
        
    # check if adapter already exists, otherwise add it
    if task_name not in model.config.adapters:
            
#             # resolve the adapter config
#             adapter_config = AdapterConfig.load(
#                 adapter_args.adapter_config,
#                 non_linearity=adapter_args.adapter_non_linearity,
#                 reduction_factor=adapter_args.adapter_reduction_factor,
#             )

        # new
        # resolve adapter config with (eventually) the MAD-X 2.0 option
        if adapter_config == "pfeiffer":
            from transformers.adapters.configuration import PfeifferConfig
            adapter_config = PfeifferConfig(non_linearity=adapter_non_linearity,
                                            reduction_factor=adapter_reduction_factor,
                                            leave_out=leave_out)           
        elif adapter_config == "pfeiffer+inv":
            from transformers.adapters.configuration import PfeifferInvConfig
            adapter_config = PfeifferInvConfig(non_linearity=adapter_non_linearity,
                                               reduction_factor=adapter_reduction_factor,
                                               leave_out=leave_out)          
        elif adapter_config == "houlsby":
            from transformers.adapters.configuration import HoulsbyConfig
            adapter_config = HoulsbyConfig(non_linearity=adapter_non_linearity,
                                           reduction_factor=adapter_reduction_factor,
                                           leave_out=leave_out)
        elif adapter_config == "houlsby+inv":
            from transformers.adapters.configuration import HoulsbyInvConfig
            adapter_config = HoulsbyInvConfig(non_linearity=adapter_non_linearity,
                                              reduction_factor=adapter_reduction_factor,
                                              leave_out=leave_out)              
                
        # load a pre-trained from Hub if specified
        if load_adapter:
            model.load_adapter(
                    load_adapter,
                    config=adapter_config,
                    load_as=task_name,
                    with_head = False
                )
        # otherwise, add a fresh adapter
        else:
            model.add_adapter(task_name, config=adapter_config)
            
    # Set the adapters to be used in every forward pass
    if lang_adapter_name:
        model.active_adapters = Stack(task_mlm_load_as, task_name)
    else:
        model.set_active_adapters(task_name)

    # Freeze all model weights except of those of this adapter
    model.train_adapter([task_name])
    
else:
    if load_adapter or load_lang_adapter:
        raise ValueError(
                "Adapters can only be loaded in adapters training mode."
                "Use --train_adapter to enable adapter training"
            )

In [88]:
model_adapter_num_param=0
for p in model.parameters():
    model_adapter_num_param+=p.numel()
model_adapter_num_param

109162525

In [91]:
print(f"Number of parameters of the model with adapter: {model_adapter_num_param:.0f}")
print(f"Number of parameters of the model without adapter: {model_num_param:.0f}")
print(f"Number of parameters of the adapter: {model_adapter_num_param - model_num_param:.0f}")
print(f"Pourcentage of additional parameters through adapter:",round(((model_adapter_num_param - model_num_param)/model_num_param)*100,2),'%')

Number of parameters of the model with adapter: 109162525
Number of parameters of the model without adapter: 108342541
Number of parameters of the adapter: 819984
Pourcentage of additional parameters through adapter: 0.76 %


## 10. Training

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [78]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    do_train=do_train,
    do_eval=do_eval,
    evaluation_strategy=evaluation_strategy,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_beta1=adam_beta1,
    adam_beta2=adam_beta2,
    adam_epsilon=adam_epsilon,
    num_train_epochs=num_train_epochs,
    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,
    warmup_steps=warmup_steps,
    logging_dir=logging_dir,         # directory for storing logs
    logging_strategy=evaluation_strategy,
    logging_steps=logging_steps,     # if strategy = "steps"
    save_strategy=evaluation_strategy,          # model checkpoint saving strategy
    save_steps=logging_steps,        # if strategy = "steps"
    save_total_limit=save_total_limit,
    fp16=fp16,
    eval_steps=logging_steps,        # if strategy = "steps"
    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,
    greater_is_better=greater_is_better,
    )

if ds:
    training_args.deepspeed = ds_config

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [79]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [80]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

In [81]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

/mnt/home/pierre/anaconda3/envs/adapter-transformers/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/home/pierre/anaconda3/envs/adapter-transformers/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/home/pierre/anaconda3/envs/adapter-transformers/lib/python3.8/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/mnt/home/pierre/anaconda3/envs/adapter-transformers/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = 

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [82]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [83]:
from transformers.trainer_callback import EarlyStoppingCallback

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"], #.shard(index=1, num_shards=50), #to be used to reduce train to 1/50
    eval_dataset=tokenized_datasets["validation"], #.shard(index=1, num_shards=50), #to be used to reduce validation to 1/50
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.args._n_gpu = n_gpu # train on one GPU
trainer.train()

In [ ]:
# save adapter + head
adapters_folder = 'adapters-' + task_name
path_to_save_adapter = path_to_outputs/adapters_folder
trainer.model.save_adapter(str(path_to_save_adapter), adapter_name=task_name, with_head=True)

!ls -lh {path_to_save_adapter}

Now, you can push the saved adapter + head to the [AdapterHub](https://adapterhub.ml/) (follow instructions at [Contributing to Adapter Hub](https://docs.adapterhub.ml/contributing.html)).

## 11. Evaluation

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

## 12. TensorBoard

In [55]:
#!pip install tensorboard

In [ ]:
import os
PATH = os.getenv('PATH')
# replace xxxx by your username on your server (ex: paulo)
# replace yyyy by the name of the virtual environment of this notebook (ex: adapter-transformers)
%env PATH=/mnt/home/xxxx/anaconda3/envs/yyyy/bin:$PATH

In [ ]:
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir {logging_dir} --bind_all

# END